In [2]:
# first test for polyphonic music
import librosa
import numpy as np
import os
from sporco.admm import cbpdn
import sporco.metric as sm
import pickle 
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.signal import argrelextrema
import scipy.io.wavfile
sr = 11025

ImportError: No module named 'librosa'

In [22]:
# load the dictionary
D = []
directory = './maps/notes/'

for i in range(21,109):
    fname1 = 'MAPS_ISOL_NO_F_S0_M' + str(i) + '_SptkBGCl.wav'
    fname2 = 'MAPS_ISOL_NO_F_S1_M' + str(i) + '_SptkBGCl.wav'
    
    try:
        s = librosa.load(directory + fname1, sr=11025,offset=0.7,duration=1.0)[0]
    except:
        s = librosa.load(directory + fname2, sr=11025,offset=0.7,duration=1.0)[0]
    # normalize dictionary elements
    D.append(s/np.amax(s))
    
D = np.asarray(D)
D = D.T

with open('mapsdict.pkl','wb') as fid:
     pickle.dump(D,fid)

In [1]:
#solve the CBPDN problem
with open('mapsdict.pkl','rb') as fid:
    D = pickle.load(fid)
print(D.shape)
lmbda = 0.005
# 1 temporal dimension
dimN = 1

opt = cbpdn.ConvBPDN.Options({'Verbose' : True, 'MaxMainIter' : 500,
                    'HighMemSolve' : True, 'LinSolveCheck' : True,
                    'RelStopTol' : 1e-3, 'AuxVarObj' : False})

# load the song
songdir = "./maps/songs/"
songname = "MAPS_MUS-alb_esp2_SptkBGCl.wav"
song, sr = librosa.load(songdir+songname,sr=sr,duration=5.0)

# load the GT
f = open('MAPS_MUS-alb_esp2_SptkBGCl.txt', 'rb')
lines = f.readlines()[1:]
GT = {}
for row in lines:
    row = row.strip('\r\n')
    row = row.split('\t')
    if row in GT:
        GT[row].append([row[1], row[2]])
    else:
        GT[row] = [[row[1], row[2]]]



NameError: name 'pickle' is not defined

In [ ]:
# sparce coding
b = cbpdn.ConvBPDN(D, song, lmbda, opt,dimN=dimN)
X = b.solve()
X = X[:,0,0,:]
with open('maps_results_esp2.pkl', 'wb') as fid:
    pickle.dump(X, fid)

In [38]:
with open('maps_results_esp2.pkl','rb') as Y:
    Y = pickle.load(Y)
    
duration = song.shape[0]/sr

lmaxes = np.asarray([])
for row in Y:
    relex = argrelextrema(row,np.greater)[0]
    lmaxes = np.concatenate((lmaxes,row[relex]))

p75,p25 = np.percentile(lmaxes, [75,25])
# try different values for iqr
threshold = p75 + 12*(p75-p25)
# lmaxes = np.sort(lmaxes)
# l = lmaxes.shape[0]
# threshold = lmaxes[95*int(l/100)]
# need to select proper threshold value
print('threshold is', threshold)
# new array of only below threshold
lowvals = Y < threshold
Y[lowvals] = 0


threshold is 0.00183084793389
finding maxima
[38] 0.45


[38 50] 0.5


[57] 1.2000000000000002


[66] 1.4000000000000001


[50 62 66 69 81] 1.4500000000000002


[33 45] 1.9000000000000001


[38] 2.4000000000000004


[38 50] 2.45


[50] 2.5


[57] 3.1500000000000004


[50 62 66 81] 3.4000000000000004


[33 45] 3.85


[50 62 66] 4.3500000000000005


[50] 4.4


[66] 4.65


[66] 4.9


[66] 4.95




In [ ]:
# peak is OK if it occurs within [onset - 0.05, offset+0.05]
#duration of each window
window_size = 0.05
start = 0
l = int(sr*window_size)
end = l
window = 0
print('finding maxima')

correct = 0
total = 0
for i in range(int(duration/window_size)):
    row = Y[start:end,:]
    sumrow = np.sum(row,axis=0)
    relex = argrelextrema(sumrow,np.greater)[0]
    if(len(relex) != 0):
        t = window_size * window
        for note in relex:
            note += 21
            total += 1
            if note in GT:
                time_ranges = GT[note]
                for time_range in time_ranges:
                    if t < float(time_range[0]) - 0.05:
                        correct += 1
                    elif t >= float(time_range[0]) - 0.05 and t <= float(time_range[1]) + 0.05:
                            break
                            
    start = end
    end += l
    window+=1

correct_percentage = float(correct) / total
print ("Correct percentage is: " + correct_percentage)